In [1]:
from satsearch import Search
import stackstac, os, requests
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import rasterio
from distributed import LocalCluster,Client
import datetime
import dask.array as dask_array
import dask
from utils import DevNullStore,DiagnosticTimer,total_nthreads,total_ncores,total_workers,get_chunksize

import geopandas as gpd
import rioxarray
import numpy as np
import xarray as xr


In [2]:

filename = r'F:\Brazil\Brazil1.geojson'
brazil = gpd.read_file(filename)

brazil32722 = brazil.to_crs(epsg=32722)
totalbounds = brazil32722.total_bounds

In [21]:
data = 'hls'
#data = 'sentinel_2a'
# data = 'landsat8'
s3 = False

if data == 'sentinel_2a':
    band='B05'
if data == 'hls':
    band='B01'
if data == 'landsat8':
    band='B1'

In [22]:
if data == 'hls':
    #Setup NASA Credentials
    urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
    prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
               'Enter NASA Earthdata Login Password: ']
    try:
        netrcDir = os.path.expanduser(r'C:\users\rscott\.netrc')
        #netrcDir = os.path.expanduser("~/.netrc")
        netrc(netrcDir).authenticators(urs)[0]
        del netrcDir

    # Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
    except FileNotFoundError:
        if 1 == 2:
            homeDir = os.path.expanduser("~")
            Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
            Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
            Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
            del homeDir, urs, prompts

In [23]:
#netrcDir
data

'hls'

In [24]:
base_env = stackstac.DEFAULT_GDAL_ENV.updated(dict(
    GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
    GDAL_SWATH_SIZE='200000000',
    VSI_CURL_CACHE_SIZE='200000000',
    GDAL_HTTP_UNSAFESSL='YES',
    GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
    GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
))

In [25]:
if s3 and data=='hls':
    #Get NASA Temp AWS Credentials
    s3_cred = requests.get('https://lpdaac.earthdata.nasa.gov/s3credentials').json()
    s3_cred
    
    env = base_env.updated(
        dict(
            AWS_REGION='us-west-2',
            AWS_NO_SIGN_REQUEST='NO',
            AWS_REQUEST_PAYER='REQUESTER',
            AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
            AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
            AWS_SESSION_TOKEN=s3_cred['sessionToken']
        )
    )

# Default to the StackStac ~.LayeredEnv
if s3==False and data=='hls':
    env = base_env.updated(dict(AWS_NO_SIGN_REQUEST='YES'))

if s3 and (data == 'landsat8' or data == 'sentinel_2a'):
    env = base_env.updated(
        dict(
            AWS_REGION='us-west-2',
            AWS_REQUEST_PAYER='REQUESTER',
            region_name='us-west-2',
            AWS_NO_SIGN_REQUEST='YES',
        )
    )
env

LayeredEnv(
    always={'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': 'tif', 'GDAL_HTTP_MULTIRANGE': 'YES', 'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES', 'GDAL_MAX_RAW_BLOCK_CACHE_SIZE': '200000000', 'GDAL_SWATH_SIZE': '200000000', 'VSI_CURL_CACHE_SIZE': '200000000', 'GDAL_HTTP_UNSAFESSL': 'YES', 'GDAL_HTTP_COOKIEFILE': 'C:\\Users\\rscott/cookies.txt', 'GDAL_HTTP_COOKIEJAR': 'C:\\Users\\rscott/cookies.txt', 'AWS_NO_SIGN_REQUEST': 'YES'},
    open={'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR', 'VSI_CACHE': True},
    open_vrt={},
    read={'VSI_CACHE': False},
)

In [10]:
s3

False

In [11]:
def get_STAC_items(url, collection, dates, bbox):
  results = Search.search(url=url,
                      collections=collection, 
                      datetime=dates,
                      bbox=bbox)

  return(results)

def remap_s3_url(stac):
    for i,entry in enumerate(stac):
        for asset in entry['assets'].keys():
            stac[i]['assets'][asset]['href'] = stac[i]['assets'][asset]['href'].replace('https://lpdaac.earthdata.nasa.gov/',
                                                                                          '/vsis3/')
            stac[i]['assets'][asset]['href'] = stac[i]['assets'][asset]['href'].replace('https://sentinel-cogs.s3.us-west-2.amazonaws.com/',
                                                                                          '/vsis3/sentinel-cogs/')
            stac[i]['assets'][asset]['href'] = stac[i]['assets'][asset]['href'].replace('https://landsat-pds.s3.us-west-2.amazonaws.com/',
                                                                                        '/vsis3/landsat-pds/')
    return(stac)

bbox = [-104.79107047,   40.78311181, -104.67687336,   40.87008987]
bbox = [-53.0172669999999968,-9.5331669999999988,-48.4956669999999974,-3.1035670000000000]

if data == 'hls':
    url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/' 
    collection = ['HLSS30.v1.5']#'C1711924822-LPCLOUD' #HLS
    dates = '2020-04-23/2021-04-23'
    stac_items = get_STAC_items(url,collection,dates,','.join(map(str, bbox))).items()
    s_col = stac_items.geojson()['features']
    if s3:
        s_col = remap_s3_url(s_col)
if data == 'sentinel_2a':
    url = 'https://earth-search.aws.element84.com/v0'
    collection = ['sentinel-s2-l2a-cogs']
    dates = '2021-03-23/2021-04-23'
    stac_items = get_STAC_items(url,collection,dates,bbox).items()
    s_col = stac_items.geojson()['features']
    if s3:
        s_col = remap_s3_url(s_col)
if data == 'landsat8':
    url = 'https://earth-search.aws.element84.com/v0'
    collection = ['landsat-8-l1-c1']
    dates = '2013-11-01/2022-02-10'
    stac_items = get_STAC_items(url,collection,dates,bbox).items()
    s_col = stac_items.geojson()['features']
    if s3:
        s_col = remap_s3_url(s_col)
print('Number of Items: ',len(s_col))

Number of Items:  424


In [12]:
with env.open, rasterio.open(s_col[0]['assets']['B07']['href']) as src:
    b= src.bounds
    prof = src.profile
    dtype = prof['dtype']
    nodata = prof['nodata']
    res = src.res
    print(prof)

NameError: name 'env' is not defined

In [39]:
#lab_extension = /user/<username>/proxy/8787/status
cluster = LocalCluster(threads_per_worker=1)
cl = Client(cluster)
cl

Client Scheduler: tcp://127.0.0.1:49167 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 127.77 GiB


In [60]:
print(b,res[0], totalbounds)

BoundingBox(left=199980.0, bottom=9290200.0, right=309780.0, top=9400000.0) 20.0 [ 277774.20749594 8946775.3372512   777108.23882963 9656553.75408149]


In [61]:
if data=='landsat8':
    band = ['B1','B2','B3','B4','B5','B6','B7']
else:
    #band=['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12']
    band=['B02']
da = stackstac.stack(s_col,dtype=dtype,
                     fill_value=nodata,
                     #resolution=res[0],
                     resolution=30,
                     epsg=32722,
                     properties=None,
                     snap_bounds=True,
                     chunksize=-1,
                     assets=band,
                     #bounds=list(b),
                     bounds=totalbounds,
                     gdal_env=env)
da

,Array,Chunk
Bytes,3.63 TiB,751.19 MiB
Shape,"(5069, 1, 23661, 16645)","(1, 1, 23661, 16645)"
Count,15208 Tasks,5069 Chunks
Type,uint16,numpy.ndarray


In [63]:
from shapely.geometry import mapping
cropped = da.rio.clip(brazil.geometry.apply(mapping), crs=4326)

In [64]:
cropped  #should be the same roughly

,Array,Chunk
Bytes,3.63 TiB,751.12 MiB
Shape,"(5069, 1, 23659, 16645)","(1, 1, 23659, 16645)"
Count,35485 Tasks,5069 Chunks
Type,uint16,numpy.ndarray


In [65]:
def median(array, dim, keep_attrs=False, skipna=True, **kwargs):
    """ Runs a median on an dask-backed xarray.
    
    This function does not scale!
    It will rechunk along the given dimension, so make sure 
    your other chunk sizes are small enough that it 
    will fit into memory.
    
    :param DataArray array: An xarray.DataArray wrapping a dask array
    :param dim str: The name of the dim in array to calculate the median
    """
    if type(array) is xr.Dataset:
        return array.apply(median, dim=dim, keep_attrs=keep_attrs, **kwargs)
    
    if not hasattr(array.data, 'dask'):
        return array.median(dim, keep_attrs=keep_attrs, **kwargs)
    
    array = array.chunk({dim:-1})
    axis = array.dims.index(dim)
    median_func = np.nanmedian if skipna else np.median
    blocks = dask.array.map_blocks(median_func, array.data, dtype=array.dtype, drop_axis=axis, axis=axis, **kwargs)
    
    new_coords={k: v for k, v in array.coords.items() if k != dim and dim not in v.dims}
    new_dims = tuple(d for d in array.dims if d != dim)
    new_attrs = array.attrs if keep_attrs else None
    
    return xr.DataArray(blocks, coords=new_coords, dims=new_dims, attrs=new_attrs)


In [71]:
import xarray as xr
import numpy as np
da = median(cropped, dim="time")


In [72]:
da

<xarray.DataArray 'nanmedian-a0302285c4b0adef651b74ba3f7cadf4' (band: 1, y: 23659, x: 16645)>
dask.array<nanmedian, shape=(1, 23659, 16645), dtype=uint16, chunksize=(1, 23659, 16645), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 9.657e+06 9.657e+06 ... 8.947e+06 8.947e+06
  * x            (x) float64 2.778e+05 2.778e+05 ... 7.771e+05 7.771e+05
  * band         (band) <U3 'B02'
    title        <U13 'Band 2 (blue)'
    proj:shape   object {10980}
    spatial_ref  int32 0

In [73]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
diag_timer = DiagnosticTimer()
devnull = DevNullStore()

dat = da.data
chunksize = get_chunksize(dat)
totalsize = dat.nbytes*1e-9

if s3:
    method='s3'
else:
    method='http'

diag_kwargs = dict(nbytes=dat.nbytes,
                   nGBytes=totalsize,
                   chunksize=chunksize,
                   method=method,
                   cloud_source='aws us-west-2',
                   system='aws us-west-2',
                   format='cog')

runtime = datetime.datetime.now().strftime("%Y%m%d_%H%M")
with diag_timer.time(nthreads=total_nthreads(cl),
                     ncores=total_ncores(cl),
                     nworkers=total_workers(cl),
                     **diag_kwargs):
    future = dask_array.store(dat, devnull, lock=False, compute=False)
    dask.compute(future, retries=5)
df = diag_timer.dataframe()
df['throughput_MBps'] = df.nbytes/1e6/df.runtime
df

KilledWorker: ("('asset-table-5170c5c95f0ca15e36a7a20486744e4b', 4184, 0)", <Worker 'tcp://127.0.0.1:49263', name: 4, memory: 0, processing: 936>)

> c:\users\rscott\appdata\local\continuum\anaconda3\envs\stackstac\lib\site-packages\distributed\client.py(1840)_gather()
   1838                             exc = CancelledError(key)
   1839                         else:
-> 1840                             raise exception.with_traceback(traceback)
   1841                         raise exc
   1842                     if errors == "skip":

